In [1]:
import nltk,string,gensim
import json
import pandas as pd


In [2]:
nltk.download('abc')

[nltk_data] Error loading abc: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>


False

In [3]:
from nltk.corpus import abc


In [4]:
#nearbt 29k sentences are available in the liabrary we can use this
sents = abc.sents()

In [5]:
len(sents)

29059

In [6]:
model = gensim.models.Word2Vec(abc.sents())

In [7]:
data = model.wv.most_similar('science')
print(data)

[('law', 0.9220589995384216), ('policy', 0.9110901355743408), ('general', 0.9042730927467346), ('discussion', 0.9033832550048828), ('media', 0.9009779095649719), ('agriculture', 0.8990622758865356), ('Sam', 0.8937610983848572), ('unprecedented', 0.8928165435791016), ('education', 0.8923560976982117), ('astronomy', 0.8916730880737305)]


### Load The Json File

In [8]:

json_file = "intents.json"
with open(json_file,'r') as f:
    data = json.load(f)

In [9]:
data

[{'tag': 'welcome',
  'patterns': ['Hi',
   'How are you',
   'Is any one to talk?',
   'Hello',
   'hi are you available'],
  'responses': ['Hello, thanks for contacting us',
   'Good to see you here',
   ' Hi there, how may I assist you?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye', 'I will come back soon'],
  'responses': ['See you later, thanks for visiting',
   'have a great day ahead',
   'Wish you Come back again soon.']},
 {'tag': 'thankful',
  'patterns': ['Thanks for helping me',
   'Thank your guidance',
   "That's helpful and kind from you"],
  'responses': ['Happy to help!',
   'Any time!',
   'My pleasure',
   'It is my duty to help you']},
 {'tag': 'hoursopening',
  'patterns': ['What hours are you open?',
   'Tell your opening time?',
   'When are you open?',
   'Just your timing please'],
  'responses': ["We're open every day 8am-7pm",
   'Our office hours are 8am-7pm every day',
   'We open office at 8 am and close at 7 pm']},
 {'tag': 'pay

In [10]:
#converting data into data Frame

In [11]:
df = pd.DataFrame(data)

In [12]:
df

,tag,patterns,responses
0,welcome,"[Hi, How are you, Is any one to talk?, Hello, ...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"[Bye, See you later, Goodbye, I will come back...","[See you later, thanks for visiting, have a gr..."
2,thankful,"[Thanks for helping me, Thank your guidance, T...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"[What hours are you open?, Tell your opening t...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"[Can I pay using credit card?, Can I pay usin...","[We accept VISA, Mastercard and credit card, W..."


In [13]:
df['patterns'] = df['patterns'].apply(', '.join)

In [14]:
df

,tag,patterns,responses
0,welcome,"Hi, How are you, Is any one to talk?, Hello, h...","[Hello, thanks for contacting us, Good to see ..."
1,goodbye,"Bye, See you later, Goodbye, I will come back ...","[See you later, thanks for visiting, have a gr..."
2,thankful,"Thanks for helping me, Thank your guidance, Th...","[Happy to help!, Any time!, My pleasure, It is..."
3,hoursopening,"What hours are you open?, Tell your opening ti...","[We're open every day 8am-7pm, Our office hour..."
4,payments,"Can I pay using credit card?, Can I pay using...","[We accept VISA, Mastercard and credit card, W..."


`!pip install textblob`

In [15]:
#preproccesig of the data 
from nltk.corpus import stopwords
from textblob import Word 
from nltk.tokenize import word_tokenize

In [16]:
stop = stopwords.words('english')
# Lowercase all words in the "patterns" column
df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x.lower() for x in x.split()))

# Remove punctuation from "patterns"
df['patterns'] = df['patterns'].apply(
    lambda x: ' '.join(x for x in word_tokenize(x) if x not in string.punctuation))

# Remove non-alphanumeric characters from "patterns"
df['patterns'] = df['patterns'].str.replace('[^\w\s]','')

# Remove digits from "patterns"
df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x for x in x.split() if not x.isdigit()))

# Remove stop words from "patterns"
df['patterns'] = df['patterns'].apply(lambda x: ' '.join(x for x in x.split() if not x in stop))

# Lemmatize all words in "patterns"
df['patterns'] = df['patterns'].apply(
    lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))




In [17]:
df['patterns']

0                       hi one talk hello hi available
1                 bye see later goodbye come back soon
2        thanks helping thank guidance 's helpful kind
3       hour open tell opening time open timing please
4    pay using credit card pay using mastercard pay...
Name: patterns, dtype: object

In [18]:
from gensim.models import Word2Vec

In [19]:
# taking the outer list
bigger_list = []
# Loop through each "patterns" entry in the DataFrame
for i in df['patterns']:
    # Split the current "patterns" string into a list of words
    li = i.split()
    bigger_list.append(li)
print(bigger_list)

[['hi', 'one', 'talk', 'hello', 'hi', 'available'], ['bye', 'see', 'later', 'goodbye', 'come', 'back', 'soon'], ['thanks', 'helping', 'thank', 'guidance', "'s", 'helpful', 'kind'], ['hour', 'open', 'tell', 'opening', 'time', 'open', 'timing', 'please'], ['pay', 'using', 'credit', 'card', 'pay', 'using', 'mastercard', 'pay', 'using', 'cash']]


In [20]:
#custom data is fed to the machine for further processsing
model = Word2Vec(bigger_list,min_count=1,workers = 4)
print(model)

Word2Vec<vocab=32, vector_size=100, alpha=0.025>


In [21]:
model.save('word2vec.model') # saving the model in system
model.save('model.bin')

In [22]:
new_model = Word2Vec.load('model.bin') # load the model from local repo


In [23]:
# Extract the vocabulary (list of all words known to the model)
# by converting the key-to-index dictionary to a simple list.
vocab = list(new_model.wv.key_to_index)
vocab

['pay',
 'using',
 'open',
 'hi',
 'credit',
 'card',
 'please',
 'cash',
 'timing',
 'time',
 'opening',
 'tell',
 'hour',
 'kind',
 'helpful',
 'mastercard',
 "'s",
 'guidance',
 'helping',
 'thank',
 'soon',
 'back',
 'come',
 'thanks',
 'goodbye',
 'later',
 'bye',
 'see',
 'available',
 'hello',
 'talk',
 'one']

In [24]:
similar_words = new_model.wv.most_similar('kind')
print(similar_words)

[('credit', 0.2528848946094513), ('later', 0.14958374202251434), ('thanks', 0.1472208946943283), ('helping', 0.14262542128562927), ('using', 0.13770504295825958), ('guidance', 0.11663373559713364), ('back', 0.07196696102619171), ('talk', 0.0620444193482399), ('come', 0.0456683486700058), ('card', 0.04410775378346443)]


In [25]:
similar_words = new_model.wv.most_similar('time')
print(similar_words)

[('pay', 0.2164667844772339), ('available', 0.12651866674423218), ('hello', 0.10320200026035309), ('bye', 0.08369238674640656), ('come', 0.07771766185760498), ('one', 0.06282763928174973), ('timing', 0.04463838413357735), ('talk', 0.030229926109313965), ('thanks', 0.02999662235379219), ('helping', 0.02045464888215065)]


In [26]:
similar_words = new_model.wv.most_similar('bye')
print(similar_words)

[('credit', 0.20074497163295746), ('please', 0.17751173675060272), ('available', 0.16687217354774475), ('later', 0.11886844784021378), ('thank', 0.11027880012989044), ('thanks', 0.10125463455915451), ('cash', 0.09661099314689636), ('time', 0.08369241654872894), ('back', 0.0825704038143158), ('guidance', 0.07622209936380386)]


In [27]:
dissimilar_words = model.wv.doesnt_match(
""" see you later, thanks for visiting """.split())
print(dissimilar_words)

see
